In [46]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Dropout
from keras import Sequential

In [47]:
dataset = pd.read_csv('/content/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [48]:
dataset = dataset.iloc[:,3:]

In [49]:
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [50]:
dataset['Gender'] = dataset['Gender'].astype('category')
dataset['Gender'] = dataset['Gender'].cat.codes
dataset['Geography'] = dataset['Geography'].astype('category')
dataset['Geography'] = dataset['Geography'].cat.codes

In [51]:
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0


In [52]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  int8   
 2   Gender           10000 non-null  int8   
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), int8(2)
memory usage: 722.8 KB


In [53]:
x = dataset.iloc[:,:-1]
y = dataset.iloc[:,-1]

In [54]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_sc = pd.DataFrame(sc.fit_transform(x), columns=x.columns)
x_sc

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,-0.326221,-0.901886,-1.095988,0.293517,-1.041760,-1.225848,-0.911583,0.646092,0.970243,0.021886
1,-0.440036,1.515067,-1.095988,0.198164,-1.387538,0.117350,-0.911583,-1.547768,0.970243,0.216534
2,-1.536794,-0.901886,-1.095988,0.293517,1.032908,1.333053,2.527057,0.646092,-1.030670,0.240687
3,0.501521,-0.901886,-1.095988,0.007457,-1.387538,-1.225848,0.807737,-1.547768,-1.030670,-0.108918
4,2.063884,1.515067,-1.095988,0.388871,-1.041760,0.785728,-0.911583,0.646092,0.970243,-0.365276
...,...,...,...,...,...,...,...,...,...,...
9995,1.246488,-0.901886,0.912419,0.007457,-0.004426,-1.225848,0.807737,0.646092,-1.030670,-0.066419
9996,-1.391939,-0.901886,0.912419,-0.373958,1.724464,-0.306379,-0.911583,0.646092,0.970243,0.027988
9997,0.604988,-0.901886,-1.095988,-0.278604,0.687130,-1.225848,-0.911583,-1.547768,0.970243,-1.008643
9998,1.256835,0.306591,0.912419,0.293517,-0.695982,-0.022608,0.807737,0.646092,-1.030670,-0.125231


In [55]:
x_sc.shape

(10000, 10)

In [56]:
# split the data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_sc, y, test_size=0.2, random_state=42, stratify=y)

In [57]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=10))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 0.4364 - accuracy: 0.8149 - val_loss: 0.3875 - val_accuracy: 0.8365
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3906 - accuracy: 0.8400 - val_loss: 0.3704 - val_accuracy: 0.8410
Epoch 3/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3725 - accuracy: 0.8465 - val_loss: 0.3532 - val_accuracy: 0.8600
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3618 - accuracy: 0.8515 - val_loss: 0.3493 - val_accuracy: 0.8560
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3589 - accuracy: 0.8525 - val_loss: 0.3509 - val_accuracy: 0.8555
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3547 - accuracy: 0.8551 - val_loss: 0.3468 - val_accuracy: 0.8595
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3492 - accuracy: 0.8561 - val_loss: 0.3448 - val_accuracy: 0.8605

In [58]:
my_data = pd.read_csv('/content/new_dataset.csv')
my_data.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,50000,1,Hargrave,307,France,Female,29,3,249220,1,1,1,197712
1,50001,2,Hill,411,Spain,Female,34,4,103469,1,0,1,306719


In [59]:
my_data.shape

(10, 13)

In [60]:
my_data.shape

(10, 13)

In [61]:
my_data['Gender'] = my_data['Gender'].astype('category')
my_data['Gender'] = my_data['Gender'].cat.codes
my_data['Geography'] = my_data['Geography'].astype('category')
my_data['Geography'] = my_data['Geography'].cat.codes

In [62]:
my_data.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,50000,1,Hargrave,307,0,0,29,3,249220,1,1,1,197712
1,50001,2,Hill,411,2,0,34,4,103469,1,0,1,306719


In [64]:
my_data = my_data.iloc[:,3:]

In [65]:
my_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,307,0,0,29,3,249220,1,1,1,197712
1,411,2,0,34,4,103469,1,0,1,306719
2,607,0,0,51,5,238440,3,1,0,265503
3,611,0,0,54,6,435236,2,0,0,274158
4,780,2,0,44,1,413554,1,1,1,332795
5,321,2,1,49,2,350640,2,1,0,205199
6,672,0,1,29,3,49117,2,1,1,148786
7,477,1,0,37,5,24569,4,1,0,132806
8,543,0,1,55,6,395868,2,0,1,189743
9,734,0,1,46,7,112747,1,1,1,183346


In [66]:
new_data_sc = pd.DataFrame(sc.transform(my_data), columns=x.columns)
new_data_sc.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,-3.554415,-0.901886,-1.095988,-0.946079,-0.695982,2.768428,-0.911583,0.646092,0.970243,1.697545
1,-2.478351,1.515067,-1.095988,-0.469311,-0.350204,0.432461,-0.911583,-1.547768,0.970243,3.593068
2,-0.450383,-0.901886,-1.095988,1.151700,-0.004426,2.595656,2.527057,0.646092,-1.030670,2.876363
3,-0.408996,-0.901886,-1.095988,1.437761,0.341352,5.749727,0.807737,-1.547768,-1.030670,3.026864
4,1.339609,1.515067,-1.095988,0.484225,-1.387538,5.402227,-0.911583,0.646092,0.970243,4.046503


In [67]:
new_output = model.predict(new_data_sc)
new_output = (new_output > 0.5)
new_output

1/1 [==============================] - 0s 52ms/step


array([[False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [False]])

In [68]:
new_output = pd.DataFrame(new_output)

In [69]:
new_output

,0
0,False
1,False
2,True
3,True
4,False
5,False
6,False
7,True
8,True
9,False


In [75]:
final_result = np.where(new_output[0]==False,0,1)

In [76]:
pd.DataFrame(final_result)

,0
0,0
1,0
2,1
3,1
4,0
5,0
6,0
7,1
8,1
9,0
